In [ ]:
# Validate GPU
import torch
assert(torch.cuda.is_available())
assert(torch.cuda.device_count() > 0)
assert(torch.cuda.current_device() == 0)
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

In [ ]:
# Load dataset
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
print("Length of data in characters: ", len(text), "\n---------------------\n")
print(text[:100])

In [ ]:
# Look at all the unique characters within the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

In [ ]:
# Encoding / decoding function are just lookup tables from the string / number and vice versa
strToNum = { ch:i for i,ch in enumerate(chars) }
numToStr = { i:ch for i,ch in enumerate(chars) }
encode = lambda string : [strToNum[char] for char in string]
decode = lambda numList : ''.join([numToStr[i] for i in numList])

startString = "Hey... whats up?"
encoded = encode(startString)
decoded = decode(encoded)

print(startString)
print(encoded)
print(decoded)

In [ ]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

In [ ]:
# Separate data into train/test split
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8 # Can also be called context length
train_data[:block_size + 1] # Add plus one

In [ ]:
# Example. It learns that given some series of input, the next character will be TARGET 
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context} the target: {target}")

In [ ]:
torch.manual_seed(1337) # Add a seed to reproduce the same results as karpathy
batch_size = 4 # How many independent sequences we process in parallel
block_size = 8 # What is the maximum context length for predictions

def get_batch(split):
    # Generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1: i + block_size + 1] for i in ix])    
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)
print('----')

for b in range(batch_size): # Batch dimension
    for t in range(block_size): # Time dimension
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'When input is {context.tolist()} the target is: {target}')

In [ ]:
# Bigram language model definition. One of the simplest language models available

import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # Each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table == nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets):
        # Idx and targets are both (B, T) tensors of integers
        logits = self.token_embedding_table(idx) # (B, T, C)
        
        return logits
bigramModel = BigramLanguageModel(vocab_size)
out = m(xb, yb)
print(out.shape)
    